# 🚀 Lab DataOps: Governança e Qualidade de Dados com PySpark e Great Expectations

## Objetivos do Laboratório

Neste laboratório prático, você irá:
- Implementar **testes de qualidade de dados** automatizados com **Great Expectations**
- Aplicar as **6 dimensões da qualidade** na prática
- Criar um **pipeline DataOps** com validações profissionais
- Simular cenários reais de **governança de dados**
- Gerar **relatórios de qualidade** automatizados

### Conceitos Aplicados
- **DataOps**: Automação e monitoramento contínuo
- **Governança**: Regras, políticas e responsabilidades
- **Qualidade**: Acurácia, Completude, Consistência, Pontualidade, Unicidade e Validade
- **Great Expectations**: Framework profissional para validação de dados

## 1. Configuração do Ambiente PySpark e Great Expectations

Primeiro, vamos configurar o ambiente PySpark e Great Expectations para nosso laboratório profissional de qualidade de dados.

In [ ]:
# Instalação do Great Expectations (execute apenas uma vez)
# !pip install great-expectations

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
from datetime import datetime, timedelta
import random
import os

# Great Expectations imports
import great_expectations as gx
from great_expectations.core.batch import RuntimeBatchRequest
from great_expectations.profile.user_configurable_profiler import UserConfigurableProfiler
from great_expectations.checkpoint import SimpleCheckpoint
from great_expectations.exceptions import DataContextError

# Inicializar Spark Session
spark = SparkSession.builder \
    .appName("DataOps_Governanca_Lab_GX") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"✅ Spark Session iniciada: {spark.version}")
print(f"📊 Contexto: {spark.sparkContext.appName}")
print(f"🎯 Great Expectations versão: {gx.__version__}")

## 2. Criação de Dados de Exemplo

Vamos criar um dataset simulando dados de **clientes de e-commerce** com problemas de qualidade intencionais para demonstrar os conceitos de governança.

In [ ]:
# Dados simulados com problemas de qualidade intencionais
dados_clientes = [
    (1, "João Silva", "joao@email.com", "11999887766", "2023-01-15", "Ativo", 25, "SP"),
    (2, "Maria Santos", "maria.santos@gmail.com", "11888776655", "2023-02-20", "Ativo", 32, "RJ"),
    (3, "Pedro", "pedro@invalid", "119999", "2023-03-10", "Inativo", 150, "MG"),  # Problemas: email inválido, telefone incompleto, idade impossível
    (4, "Ana Costa", "ana@email.com", "11777665544", "2023-04-05", "Ativo", 28, "SP"),
    (1, "João Silva", "joao@email.com", "11999887766", "2023-01-15", "Ativo", 25, "SP"),  # Duplicata
    (5, "", "carlos@email.com", "11666554433", "2023-05-12", "Pendente", None, "RS"),  # Nome vazio, idade nula
    (6, "Lucia Oliveira", "lucia@email.com", "11555443322", "2022-12-01", "Ativo", 45, "BA"),
    (7, "Roberto Lima", "roberto@email.com", "11444332211", "2023-06-18", "Cancelado", 38, "PR"),  # Status não padrão
    (8, "Fernanda", None, "11333221100", "2023-07-22", "Ativo", 29, "SC"),  # Email nulo
    (9, "Marcos Pereira", "marcos@email.com", "11222110099", "2023-08-30", "Ativo", -5, "GO")  # Idade negativa
]

# Schema definido (Governança: Padrões de Dados)
schema_clientes = StructType([
    StructField("id_cliente", IntegerType(), False),
    StructField("nome", StringType(), True),
    StructField("email", StringType(), True),
    StructField("telefone", StringType(), True),
    StructField("data_cadastro", StringType(), True),
    StructField("status", StringType(), True),
    StructField("idade", IntegerType(), True),
    StructField("estado", StringType(), True)
])

df_clientes = spark.createDataFrame(dados_clientes, schema_clientes)
print("📋 Dataset de clientes criado com problemas de qualidade intencionais")
df_clientes.show()

## 3. Configuração do Great Expectations

Vamos configurar o Great Expectations para criar um **Data Context** e definir nossas **Expectativas de Qualidade**.

In [ ]:
# Configurar diretório do Great Expectations
project_dir = "/tmp/gx_lab_dataops"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

# Inicializar Data Context do Great Expectations
try:
    context = gx.get_context()
    print("📁 Contexto Great Expectations existente carregado")
except DataContextError:
    context = gx.get_context(project_root_dir=project_dir)
    print("📁 Novo contexto Great Expectations criado")

# Converter DataFrame Spark para Pandas para Great Expectations
df_clientes_pandas = df_clientes.toPandas()

print(f"✅ Great Expectations configurado")
print(f"📊 Dataset convertido: {len(df_clientes_pandas)} registros")

## 4. Criação de Expectativas de Qualidade com Great Expectations

Vamos criar **Expectativas** (Expectations) que representam as **6 dimensões da qualidade de dados**.

In [ ]:
# Configurar Datasource para Great Expectations
datasource_config = {
    "name": "clientes_datasource",
    "class_name": "Datasource",
    "execution_engine": {
        "class_name": "PandasExecutionEngine"
    },
    "data_connectors": {
        "runtime_data_connector": {
            "class_name": "RuntimeDataConnector",
            "batch_identifiers": ["default_identifier_name"]
        }
    }
}

# Adicionar datasource ao contexto
try:
    context.add_datasource(**datasource_config)
    print("✅ Datasource 'clientes_datasource' criado")
except Exception as e:
    print(f"📋 Datasource já existe ou erro: {str(e)[:100]}...")

# Criar Expectation Suite (conjunto de expectativas)
suite_name = "clientes_quality_suite"
try:
    suite = context.create_expectation_suite(suite_name, overwrite_existing=True)
    print(f"✅ Expectation Suite '{suite_name}' criado")
except Exception as e:
    suite = context.get_expectation_suite(suite_name)
    print(f"📋 Expectation Suite '{suite_name}' já existe")

## 5. Definindo Expectativas das 6 Dimensões da Qualidade

Vamos criar expectativas específicas para cada dimensão da qualidade usando Great Expectations.

In [ ]:
# Criar batch request para os dados
batch_request = RuntimeBatchRequest(
    datasource_name="clientes_datasource",
    data_connector_name="runtime_data_connector",
    data_asset_name="clientes_dataset",
    runtime_parameters={"batch_data": df_clientes_pandas},
    batch_identifiers={"default_identifier_name": "clientes_batch"}
)

# Obter validator para criar expectativas
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=suite_name
)

print("🎯 CRIANDO EXPECTATIVAS DAS 6 DIMENSÕES DA QUALIDADE")
print("=" * 60)

# 1. COMPLETUDE (Completeness) - Campos não podem ser nulos
print("\n📊 1. COMPLETUDE - Campos obrigatórios")
validator.expect_column_values_to_not_be_null("id_cliente")
validator.expect_column_values_to_not_be_null("nome")
validator.expect_column_values_to_not_be_null("email")
print("   ✅ Expectativas de completude criadas")

# 2. UNICIDADE (Uniqueness) - Valores únicos
print("\n🔑 2. UNICIDADE - Chaves únicas")
validator.expect_column_values_to_be_unique("id_cliente")
validator.expect_column_values_to_be_unique("email")
print("   ✅ Expectativas de unicidade criadas")

# 3. VALIDADE (Validity) - Formatos e domínios
print("\n✅ 3. VALIDADE - Formatos e domínios")
# Email válido
validator.expect_column_values_to_match_regex(
    "email", 
    r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
)
# Idade válida (0-120)
validator.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
# Status válido
validator.expect_column_values_to_be_in_set("status", ["Ativo", "Inativo", "Pendente"])
print("   ✅ Expectativas de validade criadas")

# 4. CONSISTÊNCIA (Consistency) - Regras de negócio
print("\n🔄 4. CONSISTÊNCIA - Regras de negócio")
# Telefone deve ter 11 dígitos
validator.expect_column_value_lengths_to_equal("telefone", 11)
# Data de cadastro não pode ser futura
validator.expect_column_values_to_be_dateutil_parseable("data_cadastro")
print("   ✅ Expectativas de consistência criadas")

# 5. PONTUALIDADE (Timeliness) - Dados atuais
print("\n⏰ 5. PONTUALIDADE - Dados atuais")
# Data de cadastro deve ser dos últimos 2 anos
data_limite = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
validator.expect_column_values_to_be_between(
    "data_cadastro", 
    min_value=data_limite, 
    max_value=datetime.now().strftime("%Y-%m-%d")
)
print("   ✅ Expectativas de pontualidade criadas")

# 6. ACURÁCIA (Accuracy) - Dados corretos
print("\n🎯 6. ACURÁCIA - Dados corretos")
# Nome não pode ser vazio
validator.expect_column_values_to_not_match_regex("nome", r"^\s*$")
# Estado deve ser sigla válida (2 caracteres)
validator.expect_column_value_lengths_to_equal("estado", 2)
print("   ✅ Expectativas de acurácia criadas")

# Salvar expectation suite
validator.save_expectation_suite(discard_failed_expectations=False)
print("\n💾 Expectation Suite salvo com sucesso!")

## 6. Execução das Validações com Great Expectations

Agora vamos executar todas as validações e gerar um **relatório de qualidade** profissional.

In [ ]:
# Executar validações
print("🔍 EXECUTANDO VALIDAÇÕES DE QUALIDADE")
print("=" * 50)

# Validar os dados
validation_result = validator.validate()

# Analisar resultados
print(f"\n📊 RESULTADOS DA VALIDAÇÃO:")
print(f"   Total de expectativas: {validation_result.statistics['evaluated_expectations']}")
print(f"   Expectativas que passaram: {validation_result.statistics['successful_expectations']}")
print(f"   Expectativas que falharam: {validation_result.statistics['unsuccessful_expectations']}")
print(f"   Taxa de sucesso: {validation_result.statistics['success_percent']:.1f}%")

# Status geral
if validation_result.success:
    print("\n✅ VALIDAÇÃO PASSOU - Dados dentro dos padrões de qualidade")
else:
    print("\n❌ VALIDAÇÃO FALHOU - Problemas de qualidade detectados")
    print("\n🚨 PROBLEMAS ENCONTRADOS:")
    
    for result in validation_result.results:
        if not result.success:
            expectation_type = result.expectation_config.expectation_type
            column = result.expectation_config.kwargs.get('column', 'N/A')
            print(f"   ❌ {expectation_type} - Coluna: {column}")
            if 'partial_unexpected_count' in result.result:
                print(f"      Registros com problema: {result.result['partial_unexpected_count']}")

## 7. Criação de Checkpoint para Automação

Vamos criar um **Checkpoint** para automatizar as validações no pipeline DataOps.

In [ ]:
# Criar checkpoint para automação
checkpoint_name = "clientes_quality_checkpoint"

checkpoint_config = {
    "name": checkpoint_name,
    "config_version": 1.0,
    "template_name": None,
    "module_name": "great_expectations.checkpoint",
    "class_name": "Checkpoint",
    "run_name_template": "%Y%m%d-%H%M%S-clientes-validation",
    "expectation_suite_name": suite_name,
    "batch_request": {
        "datasource_name": "clientes_datasource",
        "data_connector_name": "runtime_data_connector",
        "data_asset_name": "clientes_dataset"
    },
    "action_list": [
        {
            "name": "store_validation_result",
            "action": {"class_name": "StoreValidationResultAction"},
        },
        {
            "name": "update_data_docs",
            "action": {"class_name": "UpdateDataDocsAction"},
        },
    ],
}

# Adicionar checkpoint
try:
    context.add_checkpoint(**checkpoint_config)
    print(f"✅ Checkpoint '{checkpoint_name}' criado com sucesso")
except Exception as e:
    print(f"📋 Checkpoint já existe: {str(e)[:100]}...")

print("\n🔄 EXECUTANDO CHECKPOINT AUTOMATIZADO")
print("=" * 50)

# Executar checkpoint
checkpoint_result = context.run_checkpoint(
    checkpoint_name=checkpoint_name,
    batch_request={
        "runtime_parameters": {"batch_data": df_clientes_pandas},
        "batch_identifiers": {"default_identifier_name": "clientes_batch"}
    }
)

print(f"✅ Checkpoint executado - Sucesso: {checkpoint_result.success}")

## 8. Correção de Dados e Re-validação

Vamos corrigir os problemas identificados e executar novamente as validações.

In [ ]:
def corrigir_dados_qualidade_gx(df_spark):
    """
    Aplica correções baseadas nos resultados do Great Expectations
    """
    print("🔧 APLICANDO CORREÇÕES DE QUALIDADE")
    print("=" * 50)
    
    # 1. Remover duplicatas (manter primeira ocorrência)
    df_corrigido = df_spark.dropDuplicates(["id_cliente"])
    print(f"✅ Duplicatas removidas: {df_spark.count() - df_corrigido.count()} registros")
    
    # 2. Corrigir idades inválidas
    df_corrigido = df_corrigido.withColumn(
        "idade",
        when((col("idade") < 0) | (col("idade") > 120), None)
        .otherwise(col("idade"))
    )
    print("✅ Idades inválidas corrigidas")
    
    # 3. Padronizar status
    df_corrigido = df_corrigido.withColumn(
        "status",
        when(col("status") == "Cancelado", "Inativo")
        .otherwise(col("status"))
    )
    print("✅ Status padronizados")
    
    # 4. Remover registros com campos críticos vazios
    df_corrigido = df_corrigido.filter(
        col("nome").isNotNull() & 
        (col("nome") != "") &
        col("email").isNotNull()
    )
    print("✅ Registros com campos críticos vazios removidos")
    
    print(f"\n📊 Registros após correção: {df_corrigido.count()}")
    return df_corrigido

# Aplicar correções
df_clientes_corrigido = corrigir_dados_qualidade_gx(df_clientes)
df_clientes_corrigido_pandas = df_clientes_corrigido.toPandas()

print("\n📋 Dados após correção:")
df_clientes_corrigido.show()

## 9. Re-validação com Dados Corrigidos

Vamos executar as validações novamente com os dados corrigidos.

In [ ]:
print("🔄 RE-VALIDAÇÃO COM DADOS CORRIGIDOS")
print("=" * 50)

# Executar checkpoint com dados corrigidos
checkpoint_result_corrigido = context.run_checkpoint(
    checkpoint_name=checkpoint_name,
    batch_request={
        "runtime_parameters": {"batch_data": df_clientes_corrigido_pandas},
        "batch_identifiers": {"default_identifier_name": "clientes_batch_corrigido"}
    }
)

# Comparar resultados
validation_result_corrigido = checkpoint_result_corrigido.list_validation_results()[0]

print("\n📊 COMPARAÇÃO DE RESULTADOS")
print("=" * 40)
print(f"DADOS ORIGINAIS:")
print(f"   Taxa de sucesso: {validation_result.statistics['success_percent']:.1f}%")
print(f"   Expectativas que falharam: {validation_result.statistics['unsuccessful_expectations']}")

print(f"\nDADOS CORRIGIDOS:")
print(f"   Taxa de sucesso: {validation_result_corrigido.statistics['success_percent']:.1f}%")
print(f"   Expectativas que falharam: {validation_result_corrigido.statistics['unsuccessful_expectations']}")

if validation_result_corrigido.success:
    print("\n🎉 SUCESSO! Dados corrigidos passaram em todas as validações")
else:
    print("\n⚠️ Ainda existem problemas nos dados corrigidos")

## 10. Geração de Relatórios de Qualidade

Vamos gerar **Data Docs** - relatórios HTML profissionais do Great Expectations.

In [ ]:
# Gerar Data Docs (relatórios HTML)
print("📈 GERANDO RELATÓRIOS DE QUALIDADE (DATA DOCS)")
print("=" * 50)

try:
    # Construir Data Docs
    context.build_data_docs()
    
    # Obter URLs dos relatórios
    data_docs_sites = context.get_docs_sites_urls()
    
    print("✅ Relatórios de qualidade gerados com sucesso!")
    print("\n📊 RELATÓRIOS DISPONÍVEIS:")
    
    for site_name, url in data_docs_sites.items():
        print(f"   📋 {site_name}: {url}")
        
    print("\n💡 Abra os URLs acima no navegador para visualizar os relatórios detalhados")
    
except Exception as e:
    print(f"⚠️ Erro ao gerar Data Docs: {str(e)}")

# Resumo executivo
print("\n📋 RESUMO EXECUTIVO DE QUALIDADE")
print("=" * 50)
print(f"📊 Dataset Original:")
print(f"   - Registros: {len(df_clientes_pandas)}")
print(f"   - Taxa de qualidade: {validation_result.statistics['success_percent']:.1f}%")
print(f"   - Status: {'✅ APROVADO' if validation_result.success else '❌ REPROVADO'}")

print(f"\n📊 Dataset Corrigido:")
print(f"   - Registros: {len(df_clientes_corrigido_pandas)}")
print(f"   - Taxa de qualidade: {validation_result_corrigido.statistics['success_percent']:.1f}%")
print(f"   - Status: {'✅ APROVADO' if validation_result_corrigido.success else '❌ REPROVADO'}")

melhoria = validation_result_corrigido.statistics['success_percent'] - validation_result.statistics['success_percent']
print(f"\n🎯 Melhoria alcançada: +{melhoria:.1f} pontos percentuais")

## 11. Pipeline DataOps Automatizado com Great Expectations

Vamos criar uma função que integra tudo em um **pipeline DataOps** completo.

In [ ]:
def pipeline_dataops_great_expectations(df_spark, context, checkpoint_name):
    """
    Pipeline DataOps completo usando Great Expectations
    """
    print("🚀 PIPELINE DATAOPS COM GREAT EXPECTATIONS")
    print("=" * 60)
    
    pipeline_result = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "status": "EXECUTANDO",
        "etapas_concluidas": [],
        "problemas_encontrados": [],
        "metricas_qualidade": {}
    }
    
    try:
        # Etapa 1: Validação inicial
        print("\n📊 Etapa 1: Validação inicial dos dados")
        df_pandas = df_spark.toPandas()
        
        checkpoint_result = context.run_checkpoint(
            checkpoint_name=checkpoint_name,
            batch_request={
                "runtime_parameters": {"batch_data": df_pandas},
                "batch_identifiers": {"default_identifier_name": "pipeline_batch"}
            }
        )
        
        validation_result = checkpoint_result.list_validation_results()[0]
        pipeline_result["etapas_concluidas"].append("validacao_inicial")
        pipeline_result["metricas_qualidade"]["inicial"] = validation_result.statistics
        
        print(f"   Taxa de qualidade inicial: {validation_result.statistics['success_percent']:.1f}%")
        
        # Etapa 2: Correção automática (se necessário)
        if not validation_result.success:
            print("\n🔧 Etapa 2: Aplicando correções automáticas")
            df_corrigido = corrigir_dados_qualidade_gx(df_spark)
            pipeline_result["etapas_concluidas"].append("correcao_automatica")
            
            # Etapa 3: Re-validação
            print("\n🔍 Etapa 3: Re-validação dos dados corrigidos")
            df_corrigido_pandas = df_corrigido.toPandas()
            
            checkpoint_result_final = context.run_checkpoint(
                checkpoint_name=checkpoint_name,
                batch_request={
                    "runtime_parameters": {"batch_data": df_corrigido_pandas},
                    "batch_identifiers": {"default_identifier_name": "pipeline_batch_final"}
                }
            )
            
            validation_result_final = checkpoint_result_final.list_validation_results()[0]
            pipeline_result["etapas_concluidas"].append("revalidacao")
            pipeline_result["metricas_qualidade"]["final"] = validation_result_final.statistics
            
            print(f"   Taxa de qualidade final: {validation_result_final.statistics['success_percent']:.1f}%")
            
            # Determinar status final
            if validation_result_final.success:
                pipeline_result["status"] = "✅ SUCESSO"
                print("\n🎉 PIPELINE CONCLUÍDO COM SUCESSO!")
            else:
                pipeline_result["status"] = "⚠️ SUCESSO PARCIAL"
                print("\n⚠️ Pipeline concluído com melhorias, mas ainda há problemas")
        else:
            pipeline_result["status"] = "✅ SUCESSO"
            print("\n🎉 DADOS JÁ ESTÃO EM CONFORMIDADE!")
        
        # Etapa 4: Geração de relatórios
        print("\n📈 Etapa 4: Gerando relatórios de qualidade")
        context.build_data_docs()
        pipeline_result["etapas_concluidas"].append("relatorios_gerados")
        
    except Exception as e:
        pipeline_result["status"] = "❌ ERRO"
        pipeline_result["erro"] = str(e)
        print(f"\n💥 ERRO NO PIPELINE: {e}")
    
    return pipeline_result

# Executar pipeline completo
resultado_pipeline = pipeline_dataops_great_expectations(
    df_clientes, 
    context, 
    checkpoint_name
)

print(f"\n📋 Status final do pipeline: {resultado_pipeline['status']}")
print(f"📅 Timestamp: {resultado_pipeline['timestamp']}")
print(f"✅ Etapas concluídas: {', '.join(resultado_pipeline['etapas_concluidas'])}")

## 12. Conclusões e Próximos Passos

### 🎯 O que Aprendemos com Great Expectations

Neste laboratório, implementamos um **pipeline DataOps profissional** usando Great Expectations:

#### ✅ Vantagens do Great Expectations:
- **Padronização**: Expectativas consistentes e reutilizáveis
- **Automação**: Checkpoints para execução automatizada
- **Relatórios**: Data Docs profissionais em HTML
- **Integração**: Fácil integração com pipelines de dados
- **Versionamento**: Controle de versão das expectativas

#### ✅ 6 Dimensões Implementadas:
- **Completude**: `expect_column_values_to_not_be_null`
- **Unicidade**: `expect_column_values_to_be_unique`
- **Validade**: `expect_column_values_to_match_regex`, `expect_column_values_to_be_between`
- **Consistência**: `expect_column_value_lengths_to_equal`
- **Pontualidade**: `expect_column_values_to_be_between` (datas)
- **Acurácia**: `expect_column_values_to_not_match_regex`

#### ✅ Pipeline DataOps Completo:
- **Validação automática** com checkpoints
- **Correção de dados** baseada em resultados
- **Re-validação** para confirmar melhorias
- **Relatórios automáticos** para stakeholders

### 🚀 Próximos Passos

Para evoluir este laboratório:

1. **Integração com Orquestradores**:
   - Apache Airflow com Great Expectations
   - Prefect para workflows complexos

2. **Alertas Avançados**:
   - Slack/Teams para notificações
   - Email automático para Data Stewards

3. **Métricas Avançadas**:
   - Profiling automático de dados
   - Detecção de drift de dados

### 💡 Lições Principais

> **"Great Expectations transforma validações ad-hoc em um sistema profissional de qualidade"**

- **Expectativas são código**: Versionáveis e testáveis
- **Automação é fundamental**: Checkpoints eliminam trabalho manual
- **Relatórios são essenciais**: Data Docs facilitam comunicação
- **Integração é chave**: Great Expectations funciona com qualquer pipeline

## 🧹 Limpeza do Ambiente

In [ ]:
# Finalizar Spark Session
spark.stop()
print("✅ Spark Session finalizada")
print("🎓 Laboratório Great Expectations concluído com sucesso!")
print("\n📚 Continue explorando Great Expectations para DataOps profissional!")
print("\n🔗 Recursos adicionais:")
print("   - Documentação: https://docs.greatexpectations.io/")
print("   - Galeria de Expectativas: https://greatexpectations.io/expectations/")
print("   - Comunidade: https://greatexpectations.io/community/")